# Atividade 4

## Equipe:

* Felipe Getúlio Laranjeira do Nascimento
* Lucas Pereira Reis

## Importação das bibliotecas

In [22]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier

## Realizando a leitura do *dataset*

In [13]:
df = pd.read_csv('seeds_dataset.txt', sep='\t', header=None)
df.columns = ["area","perimeter","compactness",
              "length_kernel","width_kernel","asymmetry_coefficient",
              "length_kernel_groove","variety"]

In [16]:
df.head()

,area,perimeter,compactness,length_kernel,width_kernel,asymmetry_coefficient,length_kernel_groove,variety
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


## Relatório sobre o *dataset*
**Obs:** Será apagado depois, é apenas para visualizar informações importantes

In [23]:
pandas_profiling.ProfileReport(df)

Number of variables,8
Number of observations,210
Total Missing (%),0.0%
Total size in memory,13.2 KiB
Average record size in memory,64.4 B
Numeric,4
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,4
